In [2]:
import psycopg2
import requests
import time
import re
import hidden
import myutils
import datecompat
import dateutil.parser as parser

In [ ]:
def parsemaildate(md) :
    try:
        pdate = parser.parse(tdate)
        test_at = pdate.isoformat()
        return test_at
    except:
        return datecompat.parsemaildate(md)

secrets = hidden.secrets()

conn = psycopg2.connect(host=secrets['host'],port=secrets['port'], connect_timeout=5,
        database=secrets['database'], user=secrets['user'], password=secrets['pass'])
cur = conn.cursor()

baseurl = 'http://mbox.dr-chuck.net/sakai.devel/'

cur.execute('''CREATE TABLE IF NOT EXISTS messages
    (id SERIAL, email TEXT, sent_at TIMESTAMPTZ,
     subject TEXT, headers TEXT, body TEXT)''')

# Pick up where we left off
sql = 'SELECT max(id) FROM messages'
start = myutils.queryValue(cur, sql)
if start is None : start = 0

many = 0
count = 0
fail = 0
while True:
    if ( many < 1 ) :
        conn.commit()
        sval = input('How many messages:')
        if ( len(sval) < 1 ) : break
        many = int(sval)

    start = start + 1

    # Skip rows that are already retrieved
    sql = 'SELECT id FROM messages WHERE id=%s'
    row = myutils.queryValue(cur, sql, (start,) )
    if row is not None : continue     # Skip rows that already exist

    many = many - 1
    url = baseurl + str(start) + '/' + str(start + 1)

    text = 'None'
    try:
        # Open with a timeout of 30 seconds
        response = requests.get(url)
        text = response.text
        status = response.status_code
        if status != 200 :
            print('Error code=',status, url)
            break
    except KeyboardInterrupt:
        print('')
        print('Program interrupted by user...')
        break
    except Exception as e:
        print('Unable to retrieve or parse page',url)
        print('Error',e)
        fail = fail + 1
        if fail > 5 : break
        continue

    print(url,len(text))
    count = count + 1

    if not text.startswith('From '):
        print(text)
        print('Did not find From ')
        fail = fail + 1
        if fail > 5 : break
        continue

    pos = text.find('\n\n')
    if pos > 0 :
        hdr = text[:pos]
        body = text[pos+2:]
    else:
        print(text)
        print('Could not find break between headers and body')
        fail = fail + 1
        if fail > 5 : break
        continue

    # Accept with or without < >
    email = None
    x = re.findall('\nFrom: .* <(\S+@\S+)>\n', hdr)
    if len(x) == 1 :
        email = x[0]
        email = email.strip().lower()
        email = email.replace('<','')
    else:
        x = re.findall('\nFrom: (\S+@\S+)\n', hdr)
        if len(x) == 1 :
            email = x[0]
            email = email.strip().lower()
            email = email.replace('<','')

    sent_at = None
    y = re.findall('\nDate: .*, (.*)\n', hdr)
    if len(y) == 1 :
        tdate = y[0]
        tdate = tdate[:26]
        try:
            sent_at = parsemaildate(tdate)
        except:
            print(text)
            print('Parse fail',tdate)
            fail = fail + 1
            if fail > 5 : break
            continue

    subject = None
    z = re.findall('\nSubject: (.*)\n', hdr)
    if len(z) == 1 : subject = z[0].strip().lower()

    # Reset the fail counter
    fail = 0
    print('   ',email,sent_at,subject)
    cur.execute('''INSERT INTO Messages (id, email, sent_at, subject, headers, body)
        VALUES ( %s, %s, %s, %s, %s, %s ) ON CONFLICT DO NOTHING''',
               ( start, email, sent_at, subject, hdr, body))
    if count % 50 == 0 : conn.commit()
    if count % 100 == 0 : time.sleep(1)

conn.commit()
cur.close()

How many messages:100
http://mbox.dr-chuck.net/sakai.devel/1/2 2662
    ggolden@umich.edu 2005-12-08T23:34:30-06:00 call for participation: developers documentation
http://mbox.dr-chuck.net/sakai.devel/2/3 2434
    csev@umich.edu 2005-12-09T00:58:01-05:00 report from the austin conference:  sakai developers break into song
http://mbox.dr-chuck.net/sakai.devel/3/4 3055
    kevin.carpenter@rsmart.com 2005-12-09T09:01:49-07:00 cas and sakai 1.5
http://mbox.dr-chuck.net/sakai.devel/4/5 11721
    michael.feldstein@suny.edu 2005-12-09T09:43:12-05:00 re: lms/vle rants/comments
http://mbox.dr-chuck.net/sakai.devel/5/6 9443
    john@caret.cam.ac.uk 2005-12-09T13:32:29+00:00 re: lms/vle rants/comments
http://mbox.dr-chuck.net/sakai.devel/6/7 3586
    s-githens@northwestern.edu 2005-12-09T13:32:31-06:00 re: sakaiportallogin and presense
http://mbox.dr-chuck.net/sakai.devel/7/8 10600
    john@caret.cam.ac.uk 2005-12-09T13:42:24+00:00 re: lms/vle rants/comments
http://mbox.dr-chuck.net/sakai.devel/

http://mbox.dr-chuck.net/sakai.devel/63/64 3480
    iros@student.umass.edu 2005-12-13T21:19:04-05:00 presence tool and virtual paths...
http://mbox.dr-chuck.net/sakai.devel/64/65 21272
    masayang.mobile@gmail.com 2005-12-13T22:47:23-08:00 re: utf-8 problem on sakai 2.1 with mysql
http://mbox.dr-chuck.net/sakai.devel/65/66 3399
    lfernandez@weber.edu 2005-12-13T23:22:34-07:00 re: password forgotten feature
http://mbox.dr-chuck.net/sakai.devel/66/67 4536
    lfernandez@weber.edu 2005-12-13T23:43:16-07:00 re: password forgotten feature
http://mbox.dr-chuck.net/sakai.devel/67/68 5449
    cheryl.wogahn@yale.edu 2005-12-14T08:46:29-05:00 re: password forgotten feature
http://mbox.dr-chuck.net/sakai.devel/68/69 7980
    lfernandez@weber.edu 2005-12-14T08:49:28-07:00 re: password forgotten feature
http://mbox.dr-chuck.net/sakai.devel/69/70 3684
    clayf@bu.edu 2005-12-14T08:49:31-05:00 re: sakai 2.1 webdav
http://mbox.dr-chuck.net/sakai.devel/70/71 3905
    ggolden@umich.edu 2005-12-14T09

http://mbox.dr-chuck.net/sakai.devel/125/126 10124
    nuno@ufp.pt 2005-12-15T18:39:37+00:00 re: sectionmanager api's (nosuchbeandefinitionexception)
http://mbox.dr-chuck.net/sakai.devel/126/127 2621
    news_tmp1@edegger.com 2005-12-15T21:30:28-08:00 sakai:button_bar_item onclick attribute
http://mbox.dr-chuck.net/sakai.devel/127/128 4728
    ggolden@umich.edu 2005-12-15T22:49:00-05:00 sakai 2.1 development news
http://mbox.dr-chuck.net/sakai.devel/128/129 4053
    v.n.maijer@uva.nl 2005-12-15T23:15:36+01:00 re: sakai on oracle xe
http://mbox.dr-chuck.net/sakai.devel/129/130 3637
    jimeng@umich.edu 2005-12-16T04:52:00-05:00 maven bus error?
http://mbox.dr-chuck.net/sakai.devel/130/131 3053
    jleasia@umich.edu 2005-12-16T07:46:12-05:00 re: email address as username
http://mbox.dr-chuck.net/sakai.devel/131/132 3562
    ggolden@umich.edu 2005-12-16T08:19:26-05:00 re: email address as username
http://mbox.dr-chuck.net/sakai.devel/132/133 6089
    bkirschn@umich.edu 2005-12-16T10:48:29